In [117]:
from PIL import Image,ImageOps,ImageStat,ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow  import keras as keras
from keras import Model
from keras import layers
from keras import losses
from keras.models import Sequential
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, Conv2D, add, Concatenate
#from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import  ModelCheckpoint
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.optimizers import Adam
from keras.utils import to_categorical
import os,os.path
import sys
import math
import sys
import numpy as np
import random

psp_fileName = "D:/_SAR_Kubinka_boundaries/__FRAGM1856X.JPG._PSP"
image_fileName = "D:/_SAR_Kubinka_boundaries/__FRAGM1856X.JPG"
newMask_fileName = "D:/_SAR_Kubinka_boundaries/new_mask_class_"
res_filename_winsize = "D:/_SAR_Kubinka_boundaries/homogen_parts"
fragm_dir = "D:/_SAR_Kubinka_boundaries/fragments"
param1 = 80
#acc_file = 'D:/_SAR_Kubinka/accuracy.txt'
result_fileName = "D:/_SAR_Kubinka_boundaries/classification_result"
#marks_filename = 'D:/_SAR_Kubinka/class_marks.txt'
#img_len = 1024

img_file_name = image_fileName
#дополнять массив цветами по мере необходимости\n",
t_color = ['blue', 'red']
temple_file = Image.open(img_file_name)
colors = {}
#сборка границ прямоугольникв, из которых будут вырезаться области (если их на изображении несколько)
classRects=[]
curColor = ""
class_numb = '021'
rect_flag  = False
   
img = Image.new('RGB', temple_file.size, color=0)
draw  = ImageDraw.Draw(img)
fp = open(psp_fileName)
print("all was opened")
for k, txt in enumerate(fp):
    if k<2:
        continue
    zz, t = txt.split('=')
    if t.find('Pline') >= 0:
        x2y = []
        rect_flag = False
        continue
    if t.find('Rectangle') >= 0:
        x2y = []
        rect_flag = True
        continue
    if t.find('Pen') >= 0:
        p = t.split(',')
        curColor = p[2]
        if p[2] not in  colors:
            colors.update({p[2]: len(colors)})
   
    t = t.split(' ')
    if t[0] == '':
        if rect_flag:
            x2y.insert(1,(x2y[0][0],x2y[1][1]))
            x2y.append( (x2y[2][0], x2y[0][1]))  
        idx = colors.get(curColor) 
        if idx ==  len(classRects):
            classRects.append([x2y])
        else:
            classRects[idx].append(x2y)
        #print(classRects)   
        draw.polygon(x2y, outline = t_color[idx], fill = t_color[idx])
        continue
    x2y.append((int(t[0]), int(t[1])))
print("masks were drawen")
img.save(newMask_fileName + class_numb +".BMP")
img.close()
fp.close()

all was opened
masks were drawen


In [118]:
win_size = 16
step_len = 8
bound = math.ceil(math.sqrt(2)*win_size-win_size)
dir_name = fragm_dir
labels=[]
nmb_fragm = 0
print(bound)
im = Image.open(img_file_name)
for i in range(len(colors)):
    for j in range(len(classRects[i])):
        
        x0 = classRects[i][j][0][0]
        y0 = classRects[i][j][0][1]
        x_len =  classRects[i][j][1][1]-classRects[i][j][0][1]
        y_len =  classRects[i][j][2][0]-classRects[i][j][1][0]
        x_amount = int(x_len/step_len)
        y_amount = int(y_len/step_len)
        for k in range(x_amount):
            for l in range(y_amount):
                labels.append(i)
               
                im2=im.crop((x0 + k * step_len -3, y0+ l * step_len -3, x0 + k * step_len +3 + win_size,y0+ l * step_len +3 + win_size)) 
                imfnam=dir_name+"\\+++fragm{:05d}.jpg".format(nmb_fragm); 
                nmb_fragm+=1;
                im2.save(imfnam)
im.close()

7


In [119]:
gen_crop_img = False
nmb_fragm = 0
print(nmb_fragm)
if gen_crop_img:
    img = Image.open(img_file_name)
    width, height = img.size
    print(width, height)
    with open ("D:/_SAR_Kubinka_boundaries/list_of_fragments.txt", 'wt') as file:
        for i in range(0, width, step_len):
            for j in range(0, height, step_len):
                if(i + win_size) < width and (j + win_size) < height:
                    x2y = [(i, j), (i + win_size, j), (i + win_size, j + win_size), (i, j + win_size)]
                    im2 = img.crop((i, j, i + win_size, j + win_size))
                    imfnam = "D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm{:05d}.jpg".format(nmb_fragm)
                    file.write(imfnam + " " + str(i) + " " + str(j) + "\n")
                    nmb_fragm += 1;
                    im2.save(imfnam)
    img.close()



0


In [120]:
load_in_list = True
if load_in_list:
    img_list = []
    img_coords=[]
    flag_read = True
    with open("D:\\_SAR_Kubinka_boundaries\\list_of_fragments.txt", 'r') as file:
        for bf in file:
            bf_list = bf.split(" ")
            img_list.append(bf_list[0])
            #print(img_list)
            img_coords.append((int(bf_list[1]), int(bf_list[2])))
            #print(bf)
            
                
              
               
                
                
print(img_list)

['D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00000.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00001.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00002.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00003.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00004.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00005.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00006.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00007.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00008.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00009.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00010.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00011.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00012.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00013.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00014.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\\+++fragm00015.jpg', 'D:\\_SAR_Kubinka_boundaries\\full_img\

In [121]:
transAmount = 7;
dlin   = 0

def load_CNN_train_augment(dir_name1):
    ll=[]           #пустой список имен входных файлов JPG 
    for file in os.listdir(dir_name1):
        if file.endswith(".jpg"): ll.append(file)
    dlin   = len(ll)     
    dlin0  = int(len(ll)/2)  #будем пропускать нечетные элементы
    #dlin=int(dlin0*(var+1) )  #0 ->просто чтение, 1 ->+90, 2 ->+180, 3 ->+270 дополняем массив
    train_x = np.zeros((dlin * transAmount, win_size, win_size, 3), dtype='float32')
    print('train_x_augm.shape=',train_x.shape)
    train_z = np.zeros((dlin * transAmount, 1), dtype='float32')
    dlin = dlin * transAmount
    k=0
    for file in ll:
        im  = Image.open(dir_name+"\\"+file)
        train_x[transAmount * k] = np.array(im.crop((3,3,3 + win_size, 3 +win_size)))*1./255.    
        train_x[transAmount * k + 1] = np.array(im.rotate(45).crop((3,3,3 + win_size,3 + win_size)))*1./255.    #print(b0.shape)
        shift0 = random.randint(0,6)
        shift1 = random.randint(0,6)
        shift2 = random.randint(0,6)
        shift3 = random.randint(0,6)
        #shift0 = 3
        #shift1 = 3
        #shift2 = 3
        #shift3 = 3
        if shift0 == 0 and shift1 == 0:
            shift0 += 2
        if shift2 == 0 or shift2 == 3:    
            shift2 +=1
        if shift3 == 0 or shift3 == 3:    
            shift3 +=1    
        train_x[transAmount * k + 2] = np.array(im.crop((shift0,0,shift0 + win_size,win_size)))*1./255.    #print(b0.shape)
        
        train_x[transAmount * k + 3] = np.array(im.crop((0,shift1,win_size,shift1 + win_size)))*1./255.    #print(b0.shape)
        train_x[transAmount * k + 4] = np.array(im.rotate(90).crop((shift2,shift3,shift2 + win_size,shift3 + win_size)))*1./255.
        train_x[transAmount * k + 5] = np.array(im.rotate(180).crop((3,3,3+win_size,3+win_size)))*1./255.
        train_x[transAmount * k + 6] =np.array(im.crop((0,0,0 + win_size, 0 +win_size)))*1./255. 
        #train_x[transAmount * k + 4] = np.array(im.crop((0,8,13,21)))*1./255.   
        train_z[k * transAmount] = int(labels[k])
        train_z[transAmount * k + 1] = int(labels[k])
        train_z[transAmount * k + 3] = int(labels[k])
        train_z[transAmount * k + 2] = int(labels[k])   
        train_z[transAmount * k + 4] = int(labels[k])  
        train_z[transAmount * k + 5] = int(labels[k])  
        train_z[transAmount * k + 6] = int(labels[k])  
        k+=1
        if k==dlin: break
    temp=[]
    for i in range(len(train_z)):
        temp.append(to_categorical(train_z[i],num_classes=len(colors)))
    train_y=np.array(temp)    
    return train_x,train_y

train_x1,train_y1=load_CNN_train_augment(dir_name)

train_x_augm.shape= (5803, 16, 16, 3)


In [122]:
amount_valid = 1000
k = 0
idx_del = set()
train_x_valid = np.zeros((amount_valid, win_size, win_size, 3), dtype='float32')
train_y_valid = np.zeros((amount_valid,1,2), dtype='float32')
while k < amount_valid:
    idx = random.randint(0, train_x1.shape[0]-1)
    while idx in idx_del:
        idx = random.randint(0, train_x1.shape[0]-1)
    idx_del.add(idx)
    train_x_valid[k] = train_x1[idx]
    train_y_valid[k] = train_y1[idx]
    k += 1



idx_del = list(idx_del)
train_x_data1 = np.delete(train_x1, idx_del,0)
train_y_data1 = np.delete(train_y1, idx_del,0)

print('train_x_augm.shape=',train_y_valid.shape)
print('train_x_augm.shape=',train_x_data1.shape)
#реорганизация набора тестовых данных, чтобы была равномерность 
train_x_valid_buf = np.zeros((train_x_data1.shape[0]//2, win_size, win_size, 3), dtype='float32')
train_y_valid_buf = np.zeros((train_x_data1.shape[0]//2,1,2), dtype='float32')

for i in range(train_x_data1.shape[0]//2):
    if i%2 ==0:
        train_x_valid_buf[i] = train_x_data1[2*i]
        train_y_valid_buf[i] = train_y_data1[2*i]
np.flip( train_x_valid_buf)  
np.flip( train_y_valid_buf)  
for i in range(train_x_data1.shape[0]//2):
    if i%2 ==0:
        train_x_data1[2*i] = train_x_valid_buf[i]
        train_y_data1[2*i] = train_y_valid_buf[i]
train_x_data = np.zeros((train_x1.shape[0],win_size, win_size, 3), dtype='float32')  
train_y_data = np.zeros((train_y1.shape[0],1,2), dtype='float32')  
np.concatenate(( train_x_data1,train_x_valid),  out = train_x_data)    
np.concatenate(( train_y_data1,train_y_valid),  out = train_y_data)    

train_x_augm.shape= (1000, 1, 2)
train_x_augm.shape= (4803, 16, 16, 3)


array([[[1., 0.]],

       [[1., 0.]],

       [[1., 0.]],

       ...,

       [[0., 1.]],

       [[0., 1.]],

       [[0., 1.]]], dtype=float32)

In [123]:
num_train, height_et, width_et, depth_et = train_x_data.shape # there are 50000 training examples in CIFAR-10 
#print(num_train, height, width, depth)

In [124]:
def def_cnn_model_4(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 30#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    conv_9 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_9)
   # pool_5 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_10)
    drop_5 = Dropout(drop_prob_1)(conv_10)
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [125]:
def def_cnn_model_0(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , x1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
    x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    conv_6 =add([conv_6 , x2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , x3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [126]:
def def_cnn_model_1(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 30#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4, x1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
    x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    conv_6 =add([conv_6 , x2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , x3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [127]:
def def_cnn_model_2(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , x1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
    x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    conv_6 =add([conv_6 , x2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , x3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [128]:
def train_CNN_model_ex(model_t,train_xt,train_yt,dlin_t,msize,step1,step2,val_split):
    model_temp=model_t
    val_acc_min=0.0
    for k in range(step1):
        #model_t.fit(train_xt,train_yt,epochs=step2,verbose=0,batch_size=16,validation_split=0.15)
        hist=model_t.fit(train_xt,train_yt.squeeze(),epochs=1,verbose=0,validation_split = val_split, shuffle = True)#, callbacks=[mcp_save])
        #print('First layer weights:', model.get_weights()[0])
        vvv=hist.history['val_accuracy']
        #vvv=hist.history['val_acc']
        print(k,vvv)
        val_acc = vvv[0]
        if val_acc > val_acc_min:
            model_temp = model_t
            val_acc_min=val_acc
            hist=model_t.fit(x=train_xt,y=train_yt.squeeze(),epochs=1,verbose=1,validation_split=val_split, shuffle = True, batch_size=64)#, callbacks=[mcp_save])#, callbacks=[mcp_save])
        
    #get_CNN_stat(train_xt,train_yt,dlin_t,model_t,msize)
    #get_CNN_stat(train_xz,train_yz,dlin_t,model_t,msize)
    print('end CNN train')
    return model_temp

In [129]:
def def_cnn_model_5(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [130]:
def def_cnn_model_6(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , x1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
    x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    conv_6 =add([conv_6 , x2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , x3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    x4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_9 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_10 =add([conv_10 , x4])
    
    drop_5 = Dropout(drop_prob_1)(conv_10 )
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [131]:
def def_cnn_model_7(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs =inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [132]:
def def_cnn_model_8(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 30#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [133]:
def def_cnn_model_9(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='tanh')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='tanh')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [134]:
def def_cnn_model_10(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    conv_9 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_9)
    conv_10 =add([conv_8 , drop_4])
    #pool_5 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_10)
    drop_5 = Dropout(drop_prob_1)(conv_10)
    
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [135]:
def def_cnn_model_11(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    conv_9 = Convolution2D(conv_depth_1-0, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1-0, kernel_size, kernel_size, padding='same', activation='relu')(conv_9)
    #conv_10 =add([conv_8 , drop_4])
    #pool_5 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_10)
    drop_5 = Dropout(drop_prob_1)(conv_10)
    
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [136]:
def def_cnn_model_12(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    conv_9 = Convolution2D(conv_depth_1-0, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1-0, kernel_size, kernel_size, padding='same', activation='relu')(conv_9)
    #conv_10 =add([conv_8 , drop_4])
    #pool_5 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_10)
    drop_5 = Dropout(drop_prob_1)(conv_10)
    
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [137]:
 '''with open(res_filename_0, 'w') as file:
        file.write(str(len(colors))) 
        file.write("\n")
        file.write(str(height)) 
        file.write("\n")
        for i in range(height_re):
            for j in range(width_re):
                for k in range(len(colors)):
                    file.write(str(class_prbs[j, height_re-i-1, k] )) 
                    file.write(" ") 
                file.write(" ")     
            file.write("\n")
        file.close()   '''

'with open(res_filename_0, \'w\') as file:\n       file.write(str(len(colors))) \n       file.write("\n")\n       file.write(str(height)) \n       file.write("\n")\n       for i in range(height_re):\n           for j in range(width_re):\n               for k in range(len(colors)):\n                   file.write(str(class_prbs[j, height_re-i-1, k] )) \n                   file.write(" ") \n               file.write(" ")     \n           file.write("\n")\n       file.close()   '

In [138]:
def classify_img_by_batch(model, filename_suffix):
    temple_file = Image.open(img_file_name)
    img = Image.new('RGB', temple_file.size, color=0)
    draw = ImageDraw.Draw(img)
    width, height = img.size
    class_marks = np.zeros((width, height), dtype='int')
    batch_size_t = 32
    batch_count = 0
    batch_pack = np.zeros((batch_size_t , win_size, win_size, 3), dtype='float32')
    batch_coords = []
    idx_counter = 0
    with open(res_filename_winsize, 'w') as file:
        for i in img_list:


            if(batch_count < batch_size_t ):
                im = Image.open(i)

                #print(img_coords[idx_counter])
                batch_pack[batch_count] = np.array(im)*1./255.
                #print(batch_pack[batch_count])
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
            else:
                #pred_res = model.predict_on_batch(batch_pack)
                pred_res = model.predict(batch_pack,batch_size=batch_size_t )
                #print(pred_res)
                for j in range(pred_res.shape[0]):
                    i_idx = batch_coords[j][0]
                    j_idx = batch_coords[j][1]
                    max_idx = np.argmax(pred_res[j])
                    file.write(str(i_idx) + " " + str( height - j_idx + 3) + " " + str(max_idx) + "\n")
                    for k in range(win_size):
                        for l in range(win_size):
                            class_marks[i_idx + k, j_idx + l] = max_idx
                    draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx+ win_size), (i_idx, j_idx + win_size)], outline=t_color[max_idx], fill=t_color[max_idx])
                batch_count = 0
                batch_coords = []
                im = Image.open(i)
                #print(i)
                #print(img_coords[idx_counter])
                batch_pack[batch_count] = np.array(im)*1./255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
    file.close()  
    img.save(result_fileName + filename_suffix)
    img.close()
    return class_marks

def classify_flat(model, filename_suffix):
    temple_file = Image.open(img_file_name)
    img = Image.new('RGB', temple_file.size, color=0)
    x0 = 0
    y0 = 0
    draw  = ImageDraw.Draw(img)
    width, height = img.size
    im2 = np.zeros((1, win_size, win_size, 3), dtype='float32')
    class_marks = np.zeros((width,height), dtype='int')
    for i in range(0, width,step_len ):
        for j in range(0, height,step_len):
            if(i + win_size) < width and (j + win_size ) < height:
                x2y = [(i,j), (i + win_size, j),(i + win_size, j + win_size), (i, j + win_size)]
                im2[0] = np.array(  temple_file.crop((i, j, i + win_size,j + win_size)))*1./255.
                z = model1.predict(im2)
                m_idx = np.argmax(z[0])
                for k in range(win_size):
                    for l in range(win_size):
                        class_marks[i + k, j + l] = m_idx
                draw.polygon(x2y, outline = t_color[m_idx], fill = t_color[m_idx])

    img.save(result_fileName + filename_suffix)
    img.close()
    return class_marks

In [139]:
def classify_img_by_batch_res_UDP_modified(model, filename_suffix, res_fname_suffix):
    temple_file = Image.open(img_file_name)
    img = Image.new('RGB', temple_file.size, color=0)
    draw = ImageDraw.Draw(img)
    width, height = img.size
    class_marks = np.zeros((width, height), dtype='int')
    batch_size_t = 32
    batch_count = 0
    batch_pack = np.zeros((batch_size_t , win_size, win_size, 3), dtype='float32')
    batch_coords = []
    idx_counter = 0
    homogen_parts = {}
    with open(res_filename_winsize + res_fname_suffix, 'w') as file:
        for i in img_list:
            if(batch_count < batch_size_t ):
                im = Image.open(i)
                batch_pack[batch_count] = np.array(im)*1./255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
            else:
                pred_res = model.predict(batch_pack,batch_size=batch_size_t )
                
                for j in range(pred_res.shape[0]):
                    i_idx = batch_coords[j][0]
                    j_idx = batch_coords[j][1]
                   #print(i_idx, j_idx)
                    max_idx = np.argmax(pred_res[j])
                    #if max_idx == 0:
                       # max_idx = 1
                   # else:
                        #max_idx = 0
                    homogen_parts[(i_idx,height - j_idx+ 3)]=max_idx
                    #file.write(str(i_idx) + " " + str( height - j_idx ) + " " + str(max_idx) + "\n")
                    #for k in range(win_size):
                        #for l in range(win_size):
                            #class_marks[i_idx + k, j_idx + l] = max_idx
                    #draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx+ win_size), (i_idx, j_idx + win_size)], outline=t_color[max_idx], fill=t_color[max_idx])
                batch_count = 0
                batch_coords = []
                im = Image.open(i)
                #print(i)
                #print(img_coords[idx_counter])
                batch_pack[batch_count] = np.array(im)*1./255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
        for i in homogen_parts:
            i_idx = i[0]
            j_idx = height - i[1] + 3
            elem_amount = 0
            summ = 0
            
            if (i_idx-step_len, j_idx) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx-step_len, j_idx)]
            if (i_idx + step_len, j_idx) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx + step_len, j_idx)]
            if (i_idx, j_idx-step_len) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx, j_idx-step_len)]
            if (i_idx , j_idx+ step_len) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx , j_idx + step_len)]

            if elem_amount == 4:
                if summ == 0 :#or summ == 1:
                    homogen_parts.update({(i_idx, j_idx): 0})
                if summ == 4:
                    homogen_parts.update({(i_idx, j_idx): 1})
            if elem_amount == 3:
                if summ == 0 :
                    homogen_parts.update({(i_idx, j_idx): 0})
                if summ == 3 :
                    homogen_parts.update({(i_idx, j_idx): 1})
            
        #print(homogen_parts)
        #print( homogen_parts.get((0, 0)))
        for i in homogen_parts:
            i_idx = i[0]
            j_idx = height - i[1] + 3
            file.write(str(i_idx) + " " + str(  i[1] ) + " " + str(homogen_parts[(i_idx,  i[1])]) + "\n")
            for k in range(win_size):
                for l in range(win_size):
                    class_marks[i_idx + k, j_idx + l] = homogen_parts[(i_idx,  i[1])]
            draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx+ win_size), (i_idx, j_idx + win_size)], outline=t_color[homogen_parts[(i_idx,  i[1])]], fill=t_color[homogen_parts[(i_idx,  i[1])]])
    file.close()
    img.save(result_fileName + filename_suffix)
    img.close()
    return class_marks

In [140]:
model1=def_cnn_model_7(height_et,width_et,depth_et)
msize=16
param1 = 80
model1=train_CNN_model_ex(model1,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model1, "_7_1.BMP", "_7_1.txt")
#classify_flat(model1, "_7_2.BMP")    

0 [0.5180723071098328]
82/82 [==============================] - 0s 4ms/step - loss: 0.6927 - accuracy: 0.5207 - val_loss: 0.6922 - val_accuracy: 0.5215
1 [0.5507745146751404]
82/82 [==============================] - 0s 4ms/step - loss: 0.6845 - accuracy: 0.5596 - val_loss: 0.6701 - val_accuracy: 0.6265
2 [0.6196213364601135]
82/82 [==============================] - 0s 4ms/step - loss: 0.6509 - accuracy: 0.6266 - val_loss: 0.6293 - val_accuracy: 0.6317
3 [0.6712564826011658]
82/82 [==============================] - 0s 4ms/step - loss: 0.6325 - accuracy: 0.6463 - val_loss: 0.6141 - val_accuracy: 0.6627
4 [0.6437177062034607]
5 [0.6712564826011658]
6 [0.6660929322242737]
7 [0.6540447473526001]
8 [0.6729776263237]
82/82 [==============================] - 0s 4ms/step - loss: 0.6058 - accuracy: 0.6739 - val_loss: 0.6151 - val_accuracy: 0.6575
9 [0.6660929322242737]
10 [0.6729776263237]
11 [0.6798623204231262]
82/82 [==============================] - 0s 4ms/step - loss: 0.5853 - accuracy: 0.6

1/1 [==============================] - 0s 4ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [141]:
model=def_cnn_model_9(height_et,width_et,depth_et)
msize=16
param1 = 80
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_9.BMP","_9.txt")

0 [0.5800344347953796]
82/82 [==============================] - 0s 4ms/step - loss: 0.6925 - accuracy: 0.5153 - val_loss: 0.6912 - val_accuracy: 0.5955
1 [0.5817555785179138]
82/82 [==============================] - 0s 4ms/step - loss: 0.6774 - accuracy: 0.5835 - val_loss: 0.6526 - val_accuracy: 0.6592
2 [0.6574870944023132]
82/82 [==============================] - 0s 4ms/step - loss: 0.6556 - accuracy: 0.6189 - val_loss: 0.6329 - val_accuracy: 0.6540
3 [0.6454389095306396]
4 [0.6660929322242737]
82/82 [==============================] - 0s 4ms/step - loss: 0.6373 - accuracy: 0.6517 - val_loss: 0.6090 - val_accuracy: 0.6730
5 [0.6419965624809265]
6 [0.6815834641456604]
82/82 [==============================] - 0s 4ms/step - loss: 0.6248 - accuracy: 0.6695 - val_loss: 0.6019 - val_accuracy: 0.6885
7 [0.6678141355514526]
8 [0.6850258111953735]
82/82 [==============================] - 0s 4ms/step - loss: 0.6135 - accuracy: 0.6739 - val_loss: 0.6044 - val_accuracy: 0.6609
9 [0.68674701452255

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 1ms/step


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [142]:
model=def_cnn_model_10(height_et,width_et,depth_et)
msize=16
param1 = 80
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_10.BMP","_10.txt")


0 [0.5232357978820801]
82/82 [==============================] - 0s 4ms/step - loss: 0.6908 - accuracy: 0.5316 - val_loss: 0.6883 - val_accuracy: 0.5473
1 [0.6230636835098267]
82/82 [==============================] - 0s 4ms/step - loss: 0.6828 - accuracy: 0.5624 - val_loss: 0.6674 - val_accuracy: 0.6076
2 [0.6179001927375793]
3 [0.6333907246589661]
82/82 [==============================] - 0s 4ms/step - loss: 0.6422 - accuracy: 0.6340 - val_loss: 0.6272 - val_accuracy: 0.6231
4 [0.6781411170959473]
82/82 [==============================] - 0s 5ms/step - loss: 0.6253 - accuracy: 0.6599 - val_loss: 0.6240 - val_accuracy: 0.6489
5 [0.6919105052947998]
82/82 [==============================] - 0s 4ms/step - loss: 0.6251 - accuracy: 0.6620 - val_loss: 0.6052 - val_accuracy: 0.6678
6 [0.6884681582450867]
7 [0.6678141355514526]
8 [0.6970739960670471]
82/82 [==============================] - 0s 4ms/step - loss: 0.6111 - accuracy: 0.6727 - val_loss: 0.6064 - val_accuracy: 0.6747
9 [0.66265058517456

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 4ms/step


1/1 [==============================] - 0s 7ms/step


1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 5ms/step


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [143]:
model=def_cnn_model_10(height_et,width_et,depth_et)
msize=16
param1 = 120
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_10_1.BMP","_10_1.txt")


0 [0.5180723071098328]
82/82 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.5086 - val_loss: 0.6923 - val_accuracy: 0.5181
1 [0.5043029189109802]
2 [0.6058520078659058]
82/82 [==============================] - 0s 4ms/step - loss: 0.6725 - accuracy: 0.5933 - val_loss: 0.6547 - val_accuracy: 0.6368
3 [0.6540447473526001]
82/82 [==============================] - 0s 4ms/step - loss: 0.6563 - accuracy: 0.6183 - val_loss: 0.6687 - val_accuracy: 0.6076
4 [0.6368330717086792]
5 [0.6041308045387268]
6 [0.6024096608161926]
7 [0.6523236036300659]
8 [0.6196213364601135]
9 [0.6540447473526001]
10 [0.6196213364601135]
11 [0.6729776263237]
82/82 [==============================] - 0s 4ms/step - loss: 0.6317 - accuracy: 0.6496 - val_loss: 0.6319 - val_accuracy: 0.6454
12 [0.6437177062034607]
13 [0.650602400302887]
14 [0.6368330717086792]
15 [0.6316695213317871]
16 [0.6523236036300659]
17 [0.6333907246589661]
18 [0.6299483776092529]
19 [0.6299483776092529]
20 [0.631669521331

1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 7ms/step


1/1 [==============================] - 0s 9ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [144]:
model=def_cnn_model_11(height_et,width_et,depth_et)
msize=16
param1 = 80
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_11.BMP", "_11.txt")


0 [0.5180723071098328]
82/82 [==============================] - 0s 4ms/step - loss: 0.6927 - accuracy: 0.5146 - val_loss: 0.6917 - val_accuracy: 0.5869
1 [0.4939759075641632]
2 [0.6592082381248474]
82/82 [==============================] - 0s 5ms/step - loss: 0.6710 - accuracy: 0.5967 - val_loss: 0.6611 - val_accuracy: 0.6110
3 [0.6402753591537476]
4 [0.6385542154312134]
5 [0.6678141355514526]
82/82 [==============================] - 0s 5ms/step - loss: 0.6259 - accuracy: 0.6565 - val_loss: 0.6157 - val_accuracy: 0.6506
6 [0.6230636835098267]
7 [0.6299483776092529]
8 [0.6437177062034607]
9 [0.6351118683815002]
10 [0.6660929322242737]
11 [0.6471600532531738]
12 [0.6402753591537476]
13 [0.6351118683815002]
14 [0.6609294414520264]
15 [0.6454389095306396]
16 [0.6385542154312134]
17 [0.6385542154312134]
18 [0.6798623204231262]
82/82 [==============================] - 0s 6ms/step - loss: 0.5980 - accuracy: 0.6835 - val_loss: 0.6386 - val_accuracy: 0.6661
19 [0.6712564826011658]
20 [0.66437178

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 2ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 0s/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [145]:
model=def_cnn_model_11(height_et,width_et,depth_et)
msize=16
param1 = 120
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_11_1.BMP","_11_1.txt")


0 [0.5180723071098328]
82/82 [==============================] - 0s 5ms/step - loss: 0.6929 - accuracy: 0.5134 - val_loss: 0.6925 - val_accuracy: 0.5181
1 [0.5180723071098328]
2 [0.5645439028739929]
82/82 [==============================] - 0s 5ms/step - loss: 0.6884 - accuracy: 0.5473 - val_loss: 0.6738 - val_accuracy: 0.6196
3 [0.6230636835098267]
82/82 [==============================] - 0s 5ms/step - loss: 0.6554 - accuracy: 0.6226 - val_loss: 0.6297 - val_accuracy: 0.6661
4 [0.6643717885017395]
82/82 [==============================] - 0s 5ms/step - loss: 0.6397 - accuracy: 0.6484 - val_loss: 0.6212 - val_accuracy: 0.6627
5 [0.6764199733734131]
82/82 [==============================] - 0s 5ms/step - loss: 0.6377 - accuracy: 0.6452 - val_loss: 0.6156 - val_accuracy: 0.6971
6 [0.6660929322242737]
7 [0.6746987700462341]
8 [0.6609294414520264]
9 [0.6833046674728394]
82/82 [==============================] - 0s 5ms/step - loss: 0.6221 - accuracy: 0.6542 - val_loss: 0.6035 - val_accuracy: 0.6

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 0s/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [146]:
model=def_cnn_model_12(height_et,width_et,depth_et)
msize=16
param1 = 80
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_12.BMP","_12.txt")


0 [0.5180723071098328]
82/82 [==============================] - 0s 5ms/step - loss: 0.6929 - accuracy: 0.5134 - val_loss: 0.6925 - val_accuracy: 0.5181
1 [0.5180723071098328]
2 [0.5989673137664795]
82/82 [==============================] - 0s 5ms/step - loss: 0.6641 - accuracy: 0.6088 - val_loss: 0.6313 - val_accuracy: 0.6678
3 [0.6764199733734131]
82/82 [==============================] - 0s 5ms/step - loss: 0.6426 - accuracy: 0.6362 - val_loss: 0.6067 - val_accuracy: 0.6954
4 [0.6781411170959473]
82/82 [==============================] - 0s 5ms/step - loss: 0.6228 - accuracy: 0.6647 - val_loss: 0.6060 - val_accuracy: 0.6661
5 [0.6351118683815002]
6 [0.6437177062034607]
7 [0.6540447473526001]
8 [0.6540447473526001]
9 [0.650602400302887]
10 [0.650602400302887]
11 [0.6695352792739868]
12 [0.6488812565803528]
13 [0.6437177062034607]
14 [0.6815834641456604]
82/82 [==============================] - 0s 5ms/step - loss: 0.5875 - accuracy: 0.6965 - val_loss: 0.6072 - val_accuracy: 0.6644
15 [0.6

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 0s/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [147]:
model=def_cnn_model_12(height_et,width_et,depth_et)
msize=16
param1 = 140
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_12_1.BMP", "_12_1.txt")


0 [0.5180723071098328]
82/82 [==============================] - 0s 5ms/step - loss: 0.6923 - accuracy: 0.5153 - val_loss: 0.6912 - val_accuracy: 0.4871
1 [0.5576592087745667]
82/82 [==============================] - 0s 5ms/step - loss: 0.6677 - accuracy: 0.5988 - val_loss: 0.6562 - val_accuracy: 0.6506
2 [0.6437177062034607]
82/82 [==============================] - 0s 5ms/step - loss: 0.6461 - accuracy: 0.6375 - val_loss: 0.6221 - val_accuracy: 0.6592
3 [0.6351118683815002]
4 [0.6333907246589661]
5 [0.6557658910751343]
82/82 [==============================] - 0s 5ms/step - loss: 0.6188 - accuracy: 0.6589 - val_loss: 0.6042 - val_accuracy: 0.6454
6 [0.6454389095306396]
7 [0.6678141355514526]
82/82 [==============================] - 0s 5ms/step - loss: 0.6123 - accuracy: 0.6771 - val_loss: 0.6022 - val_accuracy: 0.6695
8 [0.6523236036300659]
9 [0.6488812565803528]
10 [0.6523236036300659]
11 [0.6781411170959473]
82/82 [==============================] - 0s 5ms/step - loss: 0.5908 - accurac

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])